 ## Example in how to use ChatModel Wrapper

In [1]:
import pandas as pd

from automated_llm_eval.chat_model import ChatModel
from automated_llm_eval.utils import ProgressBar

# Instantiate wrapper around OpenAI's API
model = ChatModel(model="gpt-3.5-turbo-1106")
# model = ChatModel(model="gpt-4-1106-preview")
model

ChatModel(sync_client=<openai.OpenAI object at 0x11f9bcbd0>, async_client=<openai.AsyncOpenAI object at 0x11f9e0390>, model='gpt-3.5-turbo-1106', temperature=0.9, top_p=0.9, max_tokens=None, n=1, seed=None)

In [2]:
# You can adjust other model settings globally for all API calls
model2 = ChatModel(model="gpt-3.5-turbo-1106", temperature=0.5, top_p=0.5, max_tokens=300, seed=42)
model2

ChatModel(sync_client=<openai.OpenAI object at 0x11f9bcbd0>, async_client=<openai.AsyncOpenAI object at 0x11f9e0390>, model='gpt-3.5-turbo-1106', temperature=0.5, top_p=0.5, max_tokens=300, n=1, seed=42)

 ### Making API calls using synchronous (blocking) client

In [3]:
# Make API call, get response message.
# Note: `output_format = "simple"`
response_message = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format="simple",
)
print(response_message)

Why did the apple break up with the banana? 
Because it couldn't find a core connection!


In [4]:
# Make API call, get original ChatCompletion object.
# Note: `output_format = None`
response = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format=None,
)
print(response)

ChatCompletion(id='chatcmpl-8JFF2SxR101cQNYFQyvrsqYyUSnK7', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Sure! Did you know that apples are great at socializing? They\'re always getting invited to "core" parties!', role='assistant', function_call=None, tool_calls=None))], created=1699597512, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=24, prompt_tokens=24, total_tokens=48))


In [5]:
# Make API call, get response packaged with input + metadata.
# Note: `output_format = "message_bundle"`
message_bundle = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format="message_bundle",
)
print(message_bundle)

MessageBundle(id='chatcmpl-8JFF46SDdfvuNptYp4asGNgRPdl52', system_message='You are a joke telling machine.', user_message='Tell me something about apples.', response_message='Why did the apple go to therapy? Because it had too many core issues!', created_time=1699597514, model='gpt-3.5-turbo-1106', total_tokens=40, prompt_tokens=24, completion_tokens=16, seed=None, temperature=0.9, top_p=0.9, max_tokens=None)


In [6]:
# Make API call, get MessageBundle as a dict.
# Note: `output_format = "message_bundle_dict"`
message_bundle_dict = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format="message_bundle_dict",
)
print(message_bundle_dict)

{'id': 'chatcmpl-8JFF63WKzk4sdG70iX0evwBr1dUaf', 'system_message': 'You are a joke telling machine.', 'user_message': 'Tell me something about apples.', 'response_message': 'Why did the apple go to school? Because it wanted to be a little bit more well-rounded!', 'created_time': 1699597516, 'model': 'gpt-3.5-turbo-1106', 'total_tokens': 44, 'prompt_tokens': 24, 'completion_tokens': 20, 'seed': None, 'temperature': 0.9, 'top_p': 0.9, 'max_tokens': None}


In [7]:
# Message bundle dict can be converted into pandas Series easily
s = pd.Series(message_bundle_dict)
s

id                              chatcmpl-8JFF63WKzk4sdG70iX0evwBr1dUaf
system_message                         You are a joke telling machine.
user_message                           Tell me something about apples.
response_message     Why did the apple go to school? Because it wan...
created_time                                                1699597516
model                                               gpt-3.5-turbo-1106
total_tokens                                                        44
prompt_tokens                                                       24
completion_tokens                                                   20
seed                                                              None
temperature                                                        0.9
top_p                                                              0.9
max_tokens                                                        None
dtype: object

In [8]:
# Multiple message bundle dicts can be converted into pandas DataFrame
# NOTE: if an API call fails, then `None` will be returned. `None` items cannot
# be directly converted into pd.DataFrame
responses = []
with ProgressBar() as p:
    for _ in p.track(range(5)):
        response = model.create_chat_completion(
            system_message="You are a joke telling machine.",
            user_message="Tell me something about apples.",
            output_format="message_bundle_dict",
            temperature=0.4,
            seed=None,
        )
        responses += [response]

df = pd.DataFrame(responses)
df

Output()

,id,system_message,user_message,response_message,created_time,model,total_tokens,prompt_tokens,completion_tokens,seed,temperature,top_p,max_tokens
0,chatcmpl-8JFF7mm14Wn2KZKyriFmBKIr9cjGz,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597517,gpt-3.5-turbo-1106,41,24,17,None,0.4,0.9,None
1,chatcmpl-8JFF7WOtyhpaDyXAwI4IgrrHdWfqN,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597517,gpt-3.5-turbo-1106,41,24,17,None,0.4,0.9,None
2,chatcmpl-8JFF9heQXGGzkLtJ8wJap4WxTJt38,You are a joke telling machine.,Tell me something about apples.,Sure! Did you hear about the apple who went to...,1699597519,gpt-3.5-turbo-1106,60,24,36,None,0.4,0.9,None
3,chatcmpl-8JFFB9vbg1owUUK9BlF4tR5ghvP3Z,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597521,gpt-3.5-turbo-1106,41,24,17,None,0.4,0.9,None
4,chatcmpl-8JFFCCectMkEIEDjqxYM3KwoAVa34,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597522,gpt-3.5-turbo-1106,41,24,17,None,0.4,0.9,None


In [9]:
# If an API call fails, this method will automatically retry and make another API call.
# By default it will retry 5 times.  We can change this value to 2.
message_bundle_dict = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format="message_bundle_dict",
    num_retries=2,
)
print(message_bundle_dict)

{'id': 'chatcmpl-8JFFDzbuR6nS9szV0jiU9RgU2tAhW', 'system_message': 'You are a joke telling machine.', 'user_message': 'Tell me something about apples.', 'response_message': "Why did the apple go to the doctor?\n\nBecause it wasn't peeling well!", 'created_time': 1699597523, 'model': 'gpt-3.5-turbo-1106', 'total_tokens': 41, 'prompt_tokens': 24, 'completion_tokens': 17, 'seed': None, 'temperature': 0.9, 'top_p': 0.9, 'max_tokens': None}


In [10]:
# The `create_chat_completion` method is syntactic sugar for `chat_completion`.
# It simply formats the message for us.
system_message = "You are a joke telling machine."
user_message = "Tell me something about apples."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

message_bundle_dict = model.chat_completion(
    messages=messages,
    output_format="message_bundle_dict",
    num_retries=2,
)
print(message_bundle_dict)

{'id': 'chatcmpl-8JFFDfCbOQYwVeHOJYejsx5ZxiLIz', 'system_message': 'You are a joke telling machine.', 'user_message': 'Tell me something about apples.', 'response_message': 'Sure! Did you hear about the apple who got into a fight? It was bruised!', 'created_time': 1699597523, 'model': 'gpt-3.5-turbo-1106', 'total_tokens': 43, 'prompt_tokens': 24, 'completion_tokens': 19, 'seed': None, 'temperature': 0.9, 'top_p': 0.9, 'max_tokens': None}


 ### Making API calls using asynchronous (non-blocking) client

 This enables concurrent API calls.  We can control the max concurrency.

 Async uses the asyncio paradigm.  We need to run an asyncio event loop to
 use these functions.
 NOTE: a jupyter notebook has an asyncio event loop running by default,
 but you need to create your own asyncio event loop in a python script

In [11]:
system_message = "You are a joke telling machine."
user_message = "Tell me something about apples."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

response = await model.async_chat_completion(messages=messages)  # noqa: F704:
response


ChatCompletion(id='chatcmpl-8JFFEHjBtLwtcBlYmBkCHMA1qNjdy', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Why did the apple go to the doctor? Because it wasn't peeling well!", role='assistant', function_call=None, tool_calls=None))], created=1699597524, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=17, prompt_tokens=24, total_tokens=41))

In [12]:
# Duplicate Messages x 10 times so that we can make 10 API calls
messages_list = [messages] * 10
messages_list

[[{'role': 'system', 'content': 'You are a joke telling machine.'},
  {'role': 'user', 'content': 'Tell me something about apples.'}],
 [{'role': 'system', 'content': 'You are a joke telling machine.'},
  {'role': 'user', 'content': 'Tell me something about apples.'}],
 [{'role': 'system', 'content': 'You are a joke telling machine.'},
  {'role': 'user', 'content': 'Tell me something about apples.'}],
 [{'role': 'system', 'content': 'You are a joke telling machine.'},
  {'role': 'user', 'content': 'Tell me something about apples.'}],
 [{'role': 'system', 'content': 'You are a joke telling machine.'},
  {'role': 'user', 'content': 'Tell me something about apples.'}],
 [{'role': 'system', 'content': 'You are a joke telling machine.'},
  {'role': 'user', 'content': 'Tell me something about apples.'}],
 [{'role': 'system', 'content': 'You are a joke telling machine.'},
  {'role': 'user', 'content': 'Tell me something about apples.'}],
 [{'role': 'system', 'content': 'You are a joke telling

In [13]:
# Use Async Chat Completions, limit to 2 concurrent API calls at any given time
responses_list = await model.async_chat_completions(  # noqa: F704
    messages_list=messages_list, num_concurrent=2, output_format="message_bundle_dict"
)

df = pd.DataFrame(responses_list)
df

Output()

,id,system_message,user_message,response_message,created_time,model,total_tokens,prompt_tokens,completion_tokens,seed,temperature,top_p,max_tokens
0,chatcmpl-8JFFFNCTfjm6VS1zIq9cQCTydKQeK,You are a joke telling machine.,Tell me something about apples.,Why did the apple break up with the orange? Be...,1699597525,gpt-3.5-turbo-1106,43,24,19,None,0.9,0.9,None
1,chatcmpl-8JFFHT8BIr6EqjaVr39ZlEtr7PtoB,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597527,gpt-3.5-turbo-1106,41,24,17,None,0.9,0.9,None
2,chatcmpl-8JFFGPzV6Ol9LlPBTA8Gww2Dk3Jw2,You are a joke telling machine.,Tell me something about apples.,Sure! Did you know that apples are actually a ...,1699597526,gpt-3.5-turbo-1106,68,24,44,None,0.9,0.9,None
3,chatcmpl-8JFFHy8V0Cc3uqSNLmxqGK995wiQS,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to therapy?\n\nBecause it...,1699597527,gpt-3.5-turbo-1106,42,24,18,None,0.9,0.9,None
4,chatcmpl-8JFFIMVSkvlhSUGExSNJdue5tbSE7,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to therapy? Because it ha...,1699597528,gpt-3.5-turbo-1106,41,24,17,None,0.9,0.9,None
5,chatcmpl-8JFFJYlNCrbqKU3jxrCNcPYgs0WeG,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor?\n\nBecause...,1699597529,gpt-3.5-turbo-1106,41,24,17,None,0.9,0.9,None
6,chatcmpl-8JFFJj4ydNqdL5z8d0DPOlC8I6jt8,You are a joke telling machine.,Tell me something about apples.,Sure! Did you hear about the apple who joined ...,1699597529,gpt-3.5-turbo-1106,47,24,23,None,0.9,0.9,None
7,chatcmpl-8JFFKc1impJk3MPV84d2oQXHIQnDk,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597530,gpt-3.5-turbo-1106,41,24,17,None,0.9,0.9,None
8,chatcmpl-8JFFKcoCma8u0rjMybygfwh4gVnmL,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597530,gpt-3.5-turbo-1106,41,24,17,None,0.9,0.9,None
9,chatcmpl-8JFFKHgZsib76yJJzWoPrLKP3hS95,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699597530,gpt-3.5-turbo-1106,41,24,17,None,0.9,0.9,None
